# Interpolate soundings
## vertically from irregularly-spaced pressure levels

with Julia 1.8
author: Simon de Szoeke

In [1]:
using Dates
using NCDatasets
using Interpolations
using PyPlot

In [41]:
# Interpolation methods
# Pressure is irregular and different for each time.

"dv converts Datetimes to numeric values. Needed for Interpolations."
dv(dt, offset=DateTime(2019,3,1)) = Dates.value(dt - offset)

# interpolation functions
function itp(x, p)
    # knot-vectors must be unique and sorted in increasing order
    ind = sortperm(p)
    ii = isfinite.(p[ind])
    interpolate( (p[ind][ii],), x[ind][ii], Gridded(Linear()) )
end
itp(ds::Dataset, var, p=ds["pressure"][:][:]) = itp(ds[var][:][:], p) # convenience method

# example:
# itp(ds, "u_wind")

itp (generic function with 3 methods)

In [15]:
# define date range
dr = DateTime(2019,3,1):Day(1):DateTime(2019,6,30)
datestamps = Dates.format.(dr, "yyyymmdd_HHMM")

122-element Vector{String}:
 "20190301_0000"
 "20190302_0000"
 "20190303_0000"
 "20190304_0000"
 "20190305_0000"
 "20190306_0000"
 "20190307_0000"
 "20190308_0000"
 "20190309_0000"
 "20190310_0000"
 "20190311_0000"
 "20190312_0000"
 "20190313_0000"
 ⋮
 "20190619_0000"
 "20190620_0000"
 "20190621_0000"
 "20190622_0000"
 "20190623_0000"
 "20190624_0000"
 "20190625_0000"
 "20190626_0000"
 "20190627_0000"
 "20190628_0000"
 "20190629_0000"
 "20190630_0000"

In [43]:
# data from NetCDF
stationname = "thrivandrum"
datestamp = Dates.format(DateTime("2019-03-24T00:00"), "yyyymmdd_HHMM") # "20190324_0000"

datapath = joinpath( "../data/igra2", stationname )
joinpath( datapath, "$(stationname)$(datestamp).nc" )

# one sounding per netcdf file
data = NCDatasets.Dataset( joinpath( datapath, "$(stationname)$(datestamp).nc" ) )

# example data - one sounding, one time
u = itp(data, "u_wind")[1000]

-1.7f0

In [34]:
p=data["pressure"][:][:]
u=data["u_wind"][:][:]
ind = sortperm(p)
filter(isfinite, p[ind])

121-element Vector{Float32}:
    7.2
    7.4
    7.5
    7.8
    8.2
    8.8
   10.0
   10.5
   10.6
   11.6
   12.0
   12.9
   15.7
    ⋮
  837.0
  850.0
  861.0
  871.0
  898.0
  899.0
  925.0
  986.0
  993.0
  999.0
 1000.0
 1003.0

In [24]:
data["u_wind"][:][:]

186-element Vector{Float32}:
  -0.0
  -1.7
  -1.7
 NaN
   0.4
   1.6
 NaN
  -2.4
  -5.4
 NaN
  -4.5
  -5.6
 NaN
   ⋮
  15.8
  14.3
  12.3
  12.5
  13.4
  13.9
  15.2
  14.8
  13.9
   6.2
  10.3
   5.8